In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import json

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

Import JSON from dataset file that is output by the C# application.

In [2]:
with open('..\Output\ExternalArgumentsDatasetWithSource.json') as json_file:
    data = np.array(json.load(json_file))
    shuffle = np.random.permutation(np.arange(len(data)))
    data = data[shuffle]
    for test_case_index in range (0, 5):
        print('Class Name:', data[test_case_index]['ClassName'])
        print('Is Flawed:', data[test_case_index]['IsFlawed'])
        print(type(data[test_case_index]['IsFlawed']))
        print('Features:', data[test_case_index]['Features'])
        print('-----------------------------------------------')

Class Name: CWE089
Is Flawed: True
<class 'bool'>
Features: 


using System.Data.OracleClient;
using System;
using System.Text;

namespace default_namespace{
    class MainClass45030{
        public static void Main(string[] args){
            string tainted_2 = null;
string tainted_3 = null;

            
                tainted_2 = args[1];
            
tainted_3 = tainted_2;
            
                if((1==1)){
                    {}
                }else if(!(1==1)){
                    
                StringBuilder text = new StringBuilder(tainted_2);
                text.Replace("&", "&amp;");
                text.Replace("'", "&apos;");
                text.Replace(@"""", "&quot;");
                text.Replace("<", "&lt;");
                text.Replace(">", "&gt;");
                tainted_3 = text.ToString();
            
                }
            
                

                string query = "SELECT * FROM Articles WHERE id="+tainted_3;
            
            


In [3]:
print(len(data))
categories = ['No Flaw', 'CWE022', 'CWE078', 'CWE089', 'CWE090', 'CWE091']
labels = []
examples = []
examples_skipped = 0

for test_case_index in range(0, len(data)):
    if data[test_case_index]['IsFlawed']:
        try:
            category_index = categories.index(data[test_case_index]['ClassName'])
            labels.append(category_index)
            examples.append(data[test_case_index]['Features']) 
        except ValueError:
            examples_skipped = examples_skipped + 1
    else:
        labels.append(0)
        examples.append(data[test_case_index]['Features'])
        
print('Label Count:', len(labels))
print('Example Count:', len(examples))
print('Examples Skipped:', examples_skipped)

30980
Label Count: 30970
Example Count: 30970
Examples Skipped: 10


In [4]:
for test_case_index in range (0, 5):
    print('Label:', labels[test_case_index])
    print('Category:', categories[labels[test_case_index]])
    print('Example:', examples[test_case_index])

Label: 3
Category: CWE089
Example: 


using System.Data.OracleClient;
using System;
using System.Text;

namespace default_namespace{
    class MainClass45030{
        public static void Main(string[] args){
            string tainted_2 = null;
string tainted_3 = null;

            
                tainted_2 = args[1];
            
tainted_3 = tainted_2;
            
                if((1==1)){
                    {}
                }else if(!(1==1)){
                    
                StringBuilder text = new StringBuilder(tainted_2);
                text.Replace("&", "&amp;");
                text.Replace("'", "&apos;");
                text.Replace(@"""", "&quot;");
                text.Replace("<", "&lt;");
                text.Replace(">", "&gt;");
                tainted_3 = text.ToString();
            
                }
            
                

                string query = "SELECT * FROM Articles WHERE id="+tainted_3;
            
            
            string connec

In [5]:
num_test = int(len(examples) * 0.2)
print("num_test:", num_test)
train_data, train_labels = examples[num_test:], labels[num_test:]
dev_data, dev_labels = examples[:num_test], labels[:num_test]
print(len(dev_data))
print(len(train_data))

num_test: 6194
6194
24776


In [6]:
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w[\w\.(),]+\b",stop_words=None)
feature_vectors = vectorizer.fit_transform(train_data)
vocabulary = vectorizer.get_feature_names()
dev_vectorizer = CountVectorizer(vocabulary=vocabulary,token_pattern=r"(?u)\b\w[\w\.(),]+\b",stop_words=None)
dev_feature_vectors = dev_vectorizer.fit_transform(dev_data)

# K Nearest Neighbors
n_neighbors = {'n_neighbors': [1, 3, 5, 7, 9, 11]}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid=n_neighbors, cv=5)
grid_search.fit(feature_vectors, train_labels)
print("K Nearest Neighbors")
print("Optimal value for k:", grid_search.best_params_)
predicted_labels = grid_search.best_estimator_.predict(dev_feature_vectors)
print("F1 score:", metrics.f1_score(dev_labels, predicted_labels, average='weighted'))
print("Accuracy: ", metrics.accuracy_score(dev_labels, predicted_labels))
print()

# Multinomial Naive Bayes
alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0] }
grid_search = GridSearchCV(MultinomialNB(), param_grid=alphas, cv=5)
grid_search.fit(feature_vectors, train_labels)
print("Multinomial Naive Bayes")
print("Optimal value for alpha:", grid_search.best_params_)
predicted_labels = grid_search.best_estimator_.predict(dev_feature_vectors)
print("F1 score:", metrics.f1_score(dev_labels, predicted_labels, average='weighted'))
print("Accuracy: ", metrics.accuracy_score(dev_labels, predicted_labels))
print()

# Logistic Regression
c_values = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 500.0, 1000.0]
c = {'C': c_values }
grid_search = GridSearchCV(LogisticRegression(), param_grid=c, cv=5)
grid_search.fit(feature_vectors, train_labels)
print("Logistic Regression")
print("Optimal value for C:", grid_search.best_params_)
predicted_labels = grid_search.best_estimator_.predict(dev_feature_vectors)
print("F1 score:", metrics.f1_score(dev_labels, predicted_labels, average='weighted'))
print("Accuracy: ", metrics.accuracy_score(dev_labels, predicted_labels))
print()

matrix = confusion_matrix(dev_labels, predicted_labels)
print(matrix)
max_errors = 0
most_confused_actual = -1
most_confused_predicted = -1
# Find the most confused digit pair
for actual in range(0, len(categories)):
    for predicted in range(0, len(categories)):
        if (actual != predicted and matrix[actual, predicted] > max_errors):
            max_errors = matrix[actual, predicted]
            most_confused_actual = actual
            most_confused_predicted = predicted
print ("Most confused pair is actual =", categories[most_confused_actual], ", predicted =", categories[most_confused_predicted])
print ("This error occurred", max_errors, "times")
# Print out examples of the confused digits
error_pair_count = 0
for index in range(0, len(dev_labels)):
    if (dev_labels[index] == most_confused_actual and predicted_labels[index] == most_confused_predicted and error_pair_count < 10):
        print(dev_data[index])
        print("-----------------------------")
        error_pair_count = error_pair_count + 1

K Nearest Neighbors
Optimal value for k: {'n_neighbors': 11}
F1 score: 0.9850560849741133
Accuracy:  0.9849854698094931



C:\Users\chris\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\chris\Anaconda3\lib\s

Multinomial Naive Bayes
Optimal value for alpha: {'alpha': 0.0}
F1 score: 0.9143475427716417
Accuracy:  0.9133031966419115

Logistic Regression
Optimal value for C: {'C': 0.1}
F1 score: 0.9934626065458309
Accuracy:  0.9933806909912819

[[2531   27    0    5    0    2]
 [   0  282    0    0    0    0]
 [   0    0  277    2    0    0]
 [   4    0    0 2377    0    0]
 [   0    0    0    0  227    0]
 [   1    0    0    0    0  459]]
Most confused pair is actual = No Flaw , predicted = CWE022
This error occurred 27 times



using System;
using System.Text.RegularExpressions;
using System.IO;

namespace default_namespace{
    class MainClass4406{
        public static void Main(string[] args){
            string tainted_2 = null;
string tainted_3 = null;

            
                tainted_2 = args[1];
            
tainted_3 = tainted_2;
            
                do{
                    
                string pattern = "^[\\.\\.\\/]+";
                Regex r = new Regex(pattern);
  

In [7]:
def build_table(vectorizer):
    feature_vectors = vectorizer.fit_transform(train_data)
    c_values = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
    c = {'C': c_values }
    grid_search = GridSearchCV(LogisticRegression(), param_grid=c, cv=5)
    grid_search.fit(feature_vectors, train_labels)
    max_values = [None]*len(grid_search.best_estimator_.coef_)
    for class_index in range(0, len(grid_search.best_estimator_.coef_)):
        top_5 = np.argsort(grid_search.best_estimator_.coef_[class_index])[-5:]
        max_values[class_index] = top_5
    feature_names = vectorizer.get_feature_names()
    #print('{:<18}'.format(newsgroups_train.target_names[class_index]), '{:<20}'.format(weight), '{:<11}'.format(feature_names[feature_index])) 
    for class_index in range(0, len(grid_search.best_estimator_.coef_)):
        for feature_index in max_values[class_index]:
            weight = grid_search.best_estimator_.coef_[class_index][feature_index]
            print('{:<19}'.format(categories[class_index]), end = '')
            print('{:<20}'.format(feature_names[feature_index]), end = '')
            print('{:< .4f}'.format(grid_search.best_estimator_.coef_[0][feature_index]), end = ' ') 
            print('{:< .4f}'.format(grid_search.best_estimator_.coef_[1][feature_index]), end = ' ') 
            print('{:< .4f}'.format(grid_search.best_estimator_.coef_[2][feature_index]), end = ' ') 
            print('{:< .4f}'.format(grid_search.best_estimator_.coef_[3][feature_index])) 
            
build_table(CountVectorizer(token_pattern=r"(?u)\b\w[\w\.(),]+\b",stop_words=None))

C:\Users\chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


No Flaw            placeholder          1.0999 -0.0128 -0.0184 -1.0139
No Flaw            checked_data         1.0999 -0.0128 -0.0184 -1.0139
No Flaw            system.text.regularexpressions.regex.regex(string)isexternal(false 1.2092 -1.0777 -0.4681 -0.0901
No Flaw            hasnumbersonlyregex  1.4976 -0.0719  0.2813 -1.5069
No Flaw            hardcoded            5.4541 -1.4427 -3.1215 -4.6122
CWE022             args                -3.2977  0.8776  1.6576  2.2930
CWE022             string.format(string,object)isexternal(false,false-0.6329  0.9059 -0.3760 -0.0234
CWE022             system.text.regularexpressions.regex.escape(string)isexternal(false-0.8945  0.9084 -0.3759 -0.0240
CWE022             file.exists(tainted_3 0.4136  0.9161 -0.8998 -0.3318
CWE022             system.io.file.exists(string)isexternal(true 0.1864  2.4642 -0.9547 -0.3457
CWE078             system.diagnostics.process.start-0.2300 -0.6871  1.0489 -0.1837
CWE078             ls                  -0.2300 -0.6871  1.0